# L3 M10.2: Monitoring Financial RAG Performance

## Learning Arc

**The Problem:** Your monitoring dashboard shows "all green"—99.9% uptime, average latency 200ms, no errors. But your compliance officer just discovered that the RAG system has been providing 18-hour-old Bloomberg data, citations are 87% inaccurate, and Material Non-Public Information (MNPI) was leaked in 3 client queries yesterday.

**The Gap:** Traditional monitoring tracks infrastructure health, but financial RAG systems require **business-critical metrics** that compliance officers, CFOs, and risk managers actually care about.

**This Module's Journey:**

1. **Understand the Six Critical Financial Metrics** (not covered by standard monitoring)
   - Citation Accuracy (target: >95%)
   - Data Staleness (varies by source: Bloomberg <5min, SEC EDGAR <24h)
   - MNPI Detection Counts (zero tolerance)
   - Query Latency (p95 <2 seconds)
   - Compliance Violation Count (zero tolerance)
   - Audit Trail Completeness (100% for SOX 404)

2. **Build a Production Monitoring System** with:
   - Real-time compliance checking (MNPI, privilege, export control)
   - 1% citation verification sampling (balancing accuracy vs. performance)
   - Stakeholder-specific dashboards (CFO, CTO, Compliance Officer)
   - Intelligent alert routing based on violation type

3. **Implement SOX 404 Compliance** with:
   - 7-year audit trail retention (S3/Glacier)
   - Complete query logging (100% coverage)
   - Automated compliance report generation

**By the end**, you'll understand why monitoring "uptime 99.9%" is meaningless in finance if your data is stale, your citations are wrong, or you're leaking MNPI—and how to build monitoring that actually prevents million-dollar compliance failures.

---

In [ ]:
# Cell 2: Environment Setup and OFFLINE Mode Guard
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path for imports
sys.path.insert(0, os.path.abspath('..'))

# Check monitoring service availability (all optional)
PROMETHEUS_ENABLED = os.getenv("PROMETHEUS_ENABLED", "false").lower() == "true"
PAGERDUTY_ENABLED = os.getenv("PAGERDUTY_ENABLED", "false").lower() == "true"
AWS_S3_ENABLED = os.getenv("AWS_S3_ENABLED", "false").lower() == "true"

print("📊 L3 M10.2: Monitoring Financial RAG Performance")
print("=" * 50)
print("\n✓ Core monitoring available (local mode)")
print("\nOptional integrations:")
print(f"  - Prometheus: {'✓ Enabled' if PROMETHEUS_ENABLED else '⚠️  Disabled (local metrics only)'}")
print(f"  - PagerDuty: {'✓ Enabled' if PAGERDUTY_ENABLED else '⚠️  Disabled (logging alerts only)'}")
print(f"  - AWS S3: {'✓ Enabled' if AWS_S3_ENABLED else '⚠️  Disabled (local audit storage)'}")
print("\n💡 All monitoring features work in local mode without external services.")
print("   To enable integrations: Set environment variables in .env file\n")

## Section 1: The Six Critical Financial Metrics

Traditional monitoring focuses on infrastructure metrics (CPU, memory, uptime). Financial RAG systems need **business-critical metrics** that directly impact compliance and risk.

### Why Standard Monitoring Fails in Finance

**Scenario:** Your Grafana dashboard shows:
- ✅ Uptime: 99.95%
- ✅ Average latency: 180ms
- ✅ Error rate: 0.01%
- ✅ Database connections: Healthy

**Hidden Reality:**
- ❌ Bloomberg data is 18 hours stale (threshold: 5 minutes)
- ❌ 87% of citations are from wrong document sections
- ❌ 3 queries leaked Material Non-Public Information (MNPI)
- ❌ Audit logs missing for 15% of queries (SOX 404 violation)

**The Problem:** Infrastructure health ≠ Business outcomes in finance

In [ ]:
# Import the monitoring system
from src.l3_m10_financial_rag_in_production import (
    FinancialRAGMonitor,
    MetricsCollector,
    DataSource,
    ComplianceViolationType
)

# Initialize monitor
monitor = FinancialRAGMonitor()

print("✓ FinancialRAGMonitor initialized")
print(f"\nMonitoring Configuration:")
print(f"  - Citation sample rate: {monitor.citation_sample_rate * 100}%")
print(f"  - MNPI keywords loaded: {len(monitor.mnpi_keywords)}")
print(f"  - Audit trail: Enabled (7-year retention)")

# Show the six critical metrics
print("\n📊 Six Critical Financial Metrics:")
print("1. Citation Accuracy (target: >95%)")
print("2. Data Staleness (varies by source)")
print("3. MNPI Detection Counts (zero tolerance)")
print("4. Query Latency (p95 <2 seconds)")
print("5. Compliance Violation Count (zero tolerance)")
print("6. Audit Trail Completeness (100% for SOX 404)")

## Section 2: Tracking RAG Queries

The `track_query()` method is the main entry point for monitoring. It:
1. Measures query latency
2. Checks for compliance violations (MNPI, privilege, export control)
3. Verifies citations (1% sampling)
4. Updates data staleness metrics
5. Creates SOX 404 audit trail entry

In [ ]:
# Example 1: Track a clean query (no violations)
result = monitor.track_query(
    query="What is Apple's latest earnings guidance?",
    response="Apple's Q4 2024 guidance indicates revenue of $90-92B, representing 5-8% YoY growth.",
    citations=[
        {"source": "SEC_EDGAR_AAPL_10Q_2024Q4", "page": 12, "quote": "Revenue guidance $90-92B"},
        {"source": "Bloomberg_AAPL", "page": 1, "quote": "EPS estimate $1.35"}
    ],
    data_sources=[DataSource.SEC_EDGAR, DataSource.BLOOMBERG]
)

print("Query tracking result:")
print(f"  Status: {result['status']}")
print(f"  Latency: {result['latency_seconds']:.3f}s")
print(f"  Compliance: {'✓ Passed' if result['compliance']['passed'] else '✗ Failed'}")
print(f"  Citations: {result['citations']}")
print(f"  Audit Log ID: {result['audit_log_id']}")

## Section 3: MNPI Detection (Material Non-Public Information)

**Regulation FD (Fair Disclosure)** requires that all material information be disclosed to all investors simultaneously. MNPI leaks can result in:
- SEC violations and fines
- Criminal charges for executives
- Severe reputation damage

The monitoring system detects MNPI using keyword matching for phrases like:
- "upcoming earnings"
- "pre-announcement"
- "confidential merger"
- "material non-public"

In [ ]:
# Example 2: Query with MNPI violation
mnpi_result = monitor.track_query(
    query="What are the upcoming earnings?",
    response="The upcoming earnings announcement will show strong growth based on confidential data.",
    citations=[],
    data_sources=[]
)

print("MNPI Detection Result:")
print(f"  Compliance Passed: {mnpi_result['compliance']['passed']}")
print(f"  Violations: {len(mnpi_result['compliance']['violations'])}")

if mnpi_result['compliance']['violations']:
    for violation in mnpi_result['compliance']['violations']:
        print(f"\n  🚨 {violation['type']}")
        print(f"     Severity: {violation['severity']}")
        if 'keyword' in violation:
            print(f"     Keyword: '{violation['keyword']}'")

# Check MNPI counter
metrics = monitor.metrics_collector.get_metrics_summary()
print(f"\nTotal MNPI detections: {metrics['mnpi_detections']}")

## Section 4: Citation Accuracy Monitoring

**The 1% Sampling Strategy:**
- Verifying all citations in real-time is expensive (latency penalty)
- 1% random sampling provides statistically significant accuracy tracking
- Target: >95% accuracy (financial liability if citations are wrong)

**Citation Validation:**
A valid citation must have:
- `source`: Document identifier
- `page`: Page number
- `quote`: Quoted text

In [ ]:
# Example 3: Citation verification with 100% sampling (for demo)
monitor_full_sample = FinancialRAGMonitor({"citation_sample_rate": 1.0})

# Valid citations
result_valid = monitor_full_sample.track_query(
    query="Tesla cash flow",
    response="Tesla's Q3 2024 operating cash flow was $3.2B",
    citations=[
        {"source": "SEC_EDGAR_TSLA_10Q", "page": 8, "quote": "OCF $3.2B"},
        {"source": "Bloomberg_TSLA", "page": 1, "quote": "FCF $2.1B"}
    ],
    data_sources=[DataSource.SEC_EDGAR]
)

print("Citation Verification (Valid):")
print(f"  Sampled: {result_valid['citations']['sampled']}")
if result_valid['citations']['sampled']:
    print(f"  Accuracy: {result_valid['citations']['accuracy']:.1f}%")
    print(f"  Total citations: {result_valid['citations']['total_citations']}")
    print(f"  Correct: {result_valid['citations']['correct_citations']}")

# Invalid citations (missing fields)
result_invalid = monitor_full_sample.track_query(
    query="Microsoft risks",
    response="Microsoft faces cybersecurity risks",
    citations=[
        {"source": "SEC_EDGAR_MSFT_10K", "page": 15, "quote": "Cybersecurity risks"},
        {"source": "Bloomberg_MSFT"}  # Missing 'page' and 'quote'
    ],
    data_sources=[DataSource.SEC_EDGAR]
)

print("\nCitation Verification (Invalid):")
if result_invalid['citations']['sampled']:
    print(f"  Accuracy: {result_invalid['citations']['accuracy']:.1f}%")
    if result_invalid['citations']['accuracy'] < 95:
        print("  ⚠️ ALERT: Citation accuracy below 95% threshold!")

## Section 5: Data Staleness Monitoring

**Why Staleness Matters in Finance:**
- Trading decisions on 18-hour-old Bloomberg data → Multi-million dollar losses
- Stale SEC filings → Regulatory compliance violations

**SLA Thresholds by Source:**
- Bloomberg Terminal: <5 minutes (real-time market data)
- SEC EDGAR: <24 hours (daily filing updates)
- Internal Models: <1 hour (proprietary analytics)
- Market Data: <5 minutes (live feeds)
- Research Reports: <24 hours (analyst updates)

In [ ]:
# Check data staleness for different sources
print("Data Staleness Checks:\n")

for source in [DataSource.BLOOMBERG, DataSource.SEC_EDGAR, DataSource.INTERNAL_MODELS]:
    staleness = monitor.check_data_staleness(source)
    
    status_icon = "✓" if staleness['status'] == 'OK' else "⚠️"
    print(f"{status_icon} {staleness['source']}")
    print(f"   Hours since update: {staleness['hours_since_update']:.2f}h")
    print(f"   SLA threshold: {staleness['sla_threshold_hours']:.2f}h")
    print(f"   Status: {staleness['status']}")
    print()

# Show current staleness metrics
metrics = monitor.metrics_collector.get_metrics_summary()
print("\nCurrent Data Staleness:")
for source, hours in metrics['data_staleness'].items():
    print(f"  {source}: {hours:.2f} hours")

## Section 6: Query Latency Monitoring (p95)

**Why p95 instead of average?**
- Average latency can hide outliers (mean = 0.3s, but 5% of queries take >10s)
- p95 latency = 95th percentile = "95% of queries are faster than this"
- Target: p95 <2 seconds for analyst workflow

**Histogram buckets:** 0.1s, 0.25s, 0.5s, 1s, 2s, 5s, 10s

In [ ]:
# Simulate 100 queries with varying latencies
import random

print("Simulating 100 queries...\n")

for i in range(100):
    # Most queries fast (0.1-1s), some slow (2-5s)
    latency = random.choice([0.1] * 30 + [0.3] * 40 + [0.8] * 20 + [1.5] * 7 + [3.0] * 3)
    monitor.metrics_collector.record_query_latency(latency)

# Calculate p95
p95_latency = monitor.metrics_collector.get_p95_latency()

print(f"Results:")
print(f"  Total queries: {len(monitor.metrics_collector.metrics['query_latencies'])}")
print(f"  p95 latency: {p95_latency:.3f}s")
print(f"  SLA threshold: 2.0s")

if p95_latency < 2.0:
    print(f"  ✓ PASS: p95 latency within SLA")
else:
    print(f"  ✗ FAIL: p95 latency exceeds SLA (alert SRE on-call)")

# Show distribution
latencies = monitor.metrics_collector.metrics['query_latencies']
print(f"\nLatency distribution:")
print(f"  Min: {min(latencies):.3f}s")
print(f"  Max: {max(latencies):.3f}s")
print(f"  Mean: {sum(latencies)/len(latencies):.3f}s")
print(f"  p95: {p95_latency:.3f}s")

## Section 7: SOX 404 Compliance & Audit Trails

**SOX 404 Requirements:**
- 100% of queries must have audit logs
- 7-year retention period (2,555 days)
- Audit logs must include:
  - Timestamp (ISO 8601)
  - Query hash (not raw query for privacy)
  - Citations count
  - Compliance status
  - Latency
  - User context

**Storage:** S3 Standard → Glacier after 90 days (cost optimization)

In [ ]:
# Track several queries to generate audit logs
sample_queries = [
    {"query": "Apple earnings", "response": "Apple Q4 revenue $90B", "citations": [{"source": "SEC", "page": 1, "quote": "Rev $90B"}]},
    {"query": "Tesla cash flow", "response": "Tesla OCF $3.2B", "citations": [{"source": "SEC", "page": 2, "quote": "OCF $3.2B"}]},
    {"query": "Microsoft risks", "response": "Cybersecurity risks", "citations": [{"source": "SEC", "page": 3, "quote": "Cyber risks"}]}
]

for sq in sample_queries:
    monitor.track_query(
        query=sq["query"],
        response=sq["response"],
        citations=sq["citations"],
        data_sources=[DataSource.SEC_EDGAR]
    )

# View audit logs
audit_logs = monitor.metrics_collector.metrics["audit_logs"]

print(f"Audit Trail Summary:")
print(f"  Total audit logs: {len(audit_logs)}")
print(f"  Retention period: 7 years")
print(f"  Completeness: 100% (SOX 404 compliant)")

print(f"\nRecent audit log entries (last 3):\n")
for log in audit_logs[-3:]:
    print(f"  ID: {log['id']}")
    print(f"  Timestamp: {log['timestamp']}")
    print(f"  Query Hash: {log['query_hash'][:16]}...")
    print(f"  Citations: {log['citations_count']}")
    print(f"  Compliance: {'✓ Passed' if log['compliance_passed'] else '✗ Failed'}")
    print(f"  Latency: {log['latency_seconds']:.3f}s")
    print()

## Section 8: Compliance Report Generation

**SOX 404 Compliance Reports** must include:
- All six critical metrics
- SLA compliance status
- Period covered (start/end dates)
- Audit trail completeness
- 7-year retention status

**Use cases:**
- Quarterly compliance reviews
- External audits
- Regulatory filings

In [ ]:
from datetime import datetime, timedelta

# Generate 30-day compliance report
report = monitor.generate_compliance_report(
    start_date=datetime.utcnow() - timedelta(days=30),
    end_date=datetime.utcnow()
)

print("SOX 404 Compliance Report")
print("=" * 50)
print(f"\nReport ID: {report['report_id']}")
print(f"Period: {report['period']['start'][:10]} to {report['period']['end'][:10]}")

print(f"\nMetrics Summary:")
for key, value in report['metrics'].items():
    if isinstance(value, dict):
        print(f"  {key}: {value}")
    else:
        print(f"  {key}: {value}")

print(f"\nSLA Compliance:")
for sla, passed in report['sla_compliance'].items():
    status = "✓ PASS" if passed else "✗ FAIL"
    print(f"  {sla}: {status}")

print(f"\nAudit Trail:")
print(f"  Completeness: {report['audit_trail_completeness']}%")
print(f"  Retention: {report['retention_status']}")

print(f"\nGenerated: {report['generated_at']}")

## Section 9: Alert Routing by Violation Type

**The Problem:** All alerts go to the same on-call engineer → alert fatigue

**The Solution:** Intelligent routing based on violation type:

| Violation Type | Route To | Response Time |
|----------------|----------|---------------|
| Data Staleness SLA Breach | Data Engineering | 15 minutes |
| MNPI Detection | Compliance Officer | Immediate |
| Citation Accuracy <95% | Data Science Lead | 1 hour |
| Infrastructure Degradation | SRE On-Call | 5 minutes |
| Critical Compliance Violation | CFO + CTO + Compliance | Immediate (all three) |

**Implementation:** PagerDuty integration with escalation policies

In [ ]:
# Demonstrate alert routing logic
def route_alert(violation_type: str) -> dict:
    """
    Route alerts based on violation type.
    
    In production, this would integrate with PagerDuty API.
    """
    routing_table = {
        "data_staleness": {
            "recipient": "data-engineering-oncall@example.com",
            "response_time": "15 minutes",
            "severity": "HIGH"
        },
        "mnpi_detection": {
            "recipient": "compliance-officer@example.com",
            "response_time": "Immediate",
            "severity": "CRITICAL"
        },
        "citation_accuracy": {
            "recipient": "data-science-lead@example.com",
            "response_time": "1 hour",
            "severity": "MEDIUM"
        },
        "infrastructure": {
            "recipient": "sre-oncall@example.com",
            "response_time": "5 minutes",
            "severity": "HIGH"
        },
        "critical_violation": {
            "recipient": ["cfo@example.com", "cto@example.com", "compliance@example.com"],
            "response_time": "Immediate",
            "severity": "CRITICAL"
        }
    }
    
    return routing_table.get(violation_type, {"recipient": "default-oncall@example.com"})

# Test alert routing
print("Alert Routing Examples:\n")

test_violations = ["data_staleness", "mnpi_detection", "critical_violation"]

for violation in test_violations:
    alert = route_alert(violation)
    print(f"Violation: {violation}")
    print(f"  → Recipient: {alert['recipient']}")
    print(f"  → Response Time: {alert['response_time']}")
    print(f"  → Severity: {alert['severity']}")
    print()

## Section 10: Complete Metrics Summary

**Bringing it all together:** View all six critical financial metrics in one dashboard

In [ ]:
# Get complete metrics summary
metrics = monitor.metrics_collector.get_metrics_summary()

print("Financial RAG Monitoring Dashboard")
print("=" * 60)

print(f"\n1. Citation Accuracy")
print(f"   Current: {metrics['citation_accuracy_percent']:.1f}%")
print(f"   Target: >95%")
status = "✓ PASS" if metrics['citation_accuracy_percent'] >= 95 else "✗ FAIL"
print(f"   Status: {status}")

print(f"\n2. Data Staleness (by source)")
for source, hours in metrics['data_staleness'].items():
    print(f"   {source}: {hours:.2f} hours")

print(f"\n3. MNPI Detection Counts")
print(f"   Total detections: {metrics['mnpi_detections']}")
print(f"   Target: 0 (zero tolerance)")
status = "✓ PASS" if metrics['mnpi_detections'] == 0 else "⚠️ VIOLATIONS DETECTED"
print(f"   Status: {status}")

print(f"\n4. Query Latency")
print(f"   p95 latency: {metrics['p95_latency_seconds']:.3f}s")
print(f"   Target: <2.0s")
status = "✓ PASS" if metrics['p95_latency_seconds'] < 2.0 else "✗ FAIL"
print(f"   Status: {status}")

print(f"\n5. Compliance Violation Count")
print(f"   Total violations: {metrics['compliance_violations']}")
print(f"   Target: 0 (zero tolerance)")
status = "✓ PASS" if metrics['compliance_violations'] == 0 else "🚨 CRITICAL"
print(f"   Status: {status}")

print(f"\n6. Audit Trail Completeness")
print(f"   Total audit logs: {metrics['audit_log_count']}")
print(f"   Total queries: {metrics['successful_queries']}")
completeness = (metrics['audit_log_count'] / metrics['successful_queries'] * 100) if metrics['successful_queries'] > 0 else 0
print(f"   Completeness: {completeness:.1f}%")
print(f"   Target: 100% (SOX 404)")
status = "✓ PASS" if completeness == 100.0 else "✗ FAIL"
print(f"   Status: {status}")

print(f"\n" + "=" * 60)
print(f"Query Statistics:")
print(f"  Successful: {metrics['successful_queries']}")
print(f"  Failed: {metrics['failed_queries']}")
total = metrics['successful_queries'] + metrics['failed_queries']
success_rate = (metrics['successful_queries'] / total * 100) if total > 0 else 0
print(f"  Success Rate: {success_rate:.1f}%")

## Section 11: Cost Analysis

**Estimated Monthly Cost Breakdown (50-user investment bank):**

| Component | Tool | Cost |
|-----------|------|------|
| Metrics Collection | Prometheus | $40 |
| Visualization | Grafana | $30 |
| Alerting | PagerDuty | $40 |
| Audit Storage (7 years) | S3/Glacier | $20 |
| **Total** | | **$130/month** |

**Cost Optimization Strategies:**
1. Use 1% citation sampling (not 100%)
2. Store query hashes, not raw queries (privacy + storage savings)
3. S3 → Glacier transition after 90 days
4. Self-hosted Prometheus instead of cloud ($40 → $0)

**ROI Calculation:**
- Cost: $130/month = $1,560/year
- Prevented MNPI violation: ~$5M+ (SEC fines + reputation)
- Prevented SOX 404 audit failure: ~$2M+ (penalties + remediation)
- **ROI: >3000x**

In [ ]:
# Cost analysis calculation
monthly_costs = {
    "Prometheus": 40,
    "Grafana": 30,
    "PagerDuty": 40,
    "S3/Glacier": 20
}

total_monthly = sum(monthly_costs.values())
annual_cost = total_monthly * 12

print("Monitoring Cost Analysis")
print("=" * 50)
print("\nMonthly Breakdown:")
for component, cost in monthly_costs.items():
    print(f"  {component}: ${cost}")

print(f"\nTotal Monthly: ${total_monthly}")
print(f"Total Annual: ${annual_cost:,}")

# ROI calculation
prevented_mnpi_fine = 5_000_000  # Conservative estimate
prevented_sox_penalty = 2_000_000
total_risk_prevented = prevented_mnpi_fine + prevented_sox_penalty

roi = (total_risk_prevented - annual_cost) / annual_cost

print(f"\nROI Analysis:")
print(f"  Annual Investment: ${annual_cost:,}")
print(f"  Risk Prevented (MNPI): ${prevented_mnpi_fine:,}")
print(f"  Risk Prevented (SOX 404): ${prevented_sox_penalty:,}")
print(f"  Total Risk Prevented: ${total_risk_prevented:,}")
print(f"\n  ROI: {roi:.0f}x return on investment")
print(f"\n💡 Spending ${annual_cost:,}/year prevents ~${total_risk_prevented:,} in violations")

## Summary

### Key Takeaways

1. **Infrastructure metrics ≠ Business outcomes** in finance
   - "99.9% uptime" is meaningless if data is stale or citations are wrong
   - Financial RAG systems need **six critical metrics** beyond standard monitoring

2. **The Six Critical Financial Metrics:**
   - Citation Accuracy (>95%)
   - Data Staleness (varies by source)
   - MNPI Detection Counts (zero tolerance)
   - Query Latency (p95 <2s)
   - Compliance Violation Count (zero tolerance)
   - Audit Trail Completeness (100% for SOX 404)

3. **Cost-Effective Implementation:**
   - ~$130/month for 50-user investment bank
   - >3000x ROI by preventing MNPI violations and SOX 404 failures
   - Use 1% citation sampling to balance accuracy monitoring with performance

4. **Intelligent Alert Routing:**
   - Different violations require different responses
   - Route MNPI detections to Compliance Officer (immediate)
   - Route data staleness to Data Engineering (15 minutes)
   - Critical violations → CFO + CTO + Compliance (all three)

5. **SOX 404 Compliance:**
   - 100% audit trail completeness required
   - 7-year retention period (S3 → Glacier)
   - Store query hashes, not raw queries (privacy + storage savings)

### What We Built

- ✅ Production-ready monitoring system for financial RAG
- ✅ Real-time compliance checking (MNPI, privilege, export control)
- ✅ Citation verification with 1% sampling
- ✅ Data staleness monitoring with source-specific SLAs
- ✅ SOX 404 compliant audit trails
- ✅ Intelligent alert routing

### Next Steps

1. **Production Deployment:**
   - Set up Prometheus + Grafana dashboards
   - Configure PagerDuty alert routing
   - Enable S3/Glacier audit storage

2. **Advanced Features:**
   - Machine learning-based MNPI detection (beyond keywords)
   - Automated citation verification using document embeddings
   - Predictive data staleness alerts

3. **Stakeholder Dashboards:**
   - CFO dashboard: Focus on compliance violations, audit trail
   - CTO dashboard: Focus on latency, infrastructure health
   - Compliance Officer dashboard: Focus on MNPI, citation accuracy

---

**Remember:** In financial AI, "all green" on the infrastructure dashboard means nothing if you're leaking MNPI, serving stale data, or providing inaccurate citations. Monitor what matters to your business stakeholders, not just your DevOps team.

**Next Module:** M10.3 - Scaling Financial RAG to Production Traffic